In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5 MB 9.0 MB/s 
     |████████████████████████████████| 895 kB 80.9 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 


In [2]:
!pip install -q datasets seqeval
!pip install -q pyyaml==5.1

     |████████████████████████████████| 325 kB 7.4 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 212 kB 89.0 MB/s 
     |████████████████████████████████| 136 kB 85.2 MB/s 
     |████████████████████████████████| 1.1 MB 69.8 MB/s 
     |████████████████████████████████| 127 kB 88.7 MB/s 
     |████████████████████████████████| 271 kB 90.2 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 144 kB 85.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 274 kB 7.8 MB/s 


In [3]:
!nvidia-smi

Mon Apr 11 02:56:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
import torch
torch.__version__

'1.10.0+cu111'

In [6]:
!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

     |████████████████████████████████| 7.0 MB 3.5 MB/s 
     |████████████████████████████████| 130 kB 8.1 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 145 kB 65.3 MB/s 
     |████████████████████████████████| 74 kB 4.5 MB/s 
     |████████████████████████████████| 749 kB 76.5 MB/s 
     |████████████████████████████████| 843 kB 42.7 MB/s 
     |████████████████████████████████| 112 kB 76.9 MB/s 


In [7]:
from datasets import load_dataset

datasets = load_dataset("nielsr/funsd")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset funsd downloaded and prepared to /root/.cache/huggingface/datasets/nielsr___funsd/funsd/1.0.0/8b0472b536a2dcb975d59a4fb9d6fea4e6a1abe260b7fed6f75301e168cbe595. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 50
    })
})

## Preprocess data

In [9]:
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)

['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER']


In [10]:
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id

{'B-ANSWER': 5,
 'B-HEADER': 1,
 'B-QUESTION': 3,
 'I-ANSWER': 6,
 'I-HEADER': 2,
 'I-QUESTION': 4,
 'O': 0}

In [11]:
from PIL import Image
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")

# We need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype="int64")),
    'attention_mask': Sequence(feature=Value(dtype="int64")),
    'token_type_ids': Sequence(feature=Value(dtype="int64")),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels))
})

def preprocess_data(examples):
    images = [Image.open(path).convert('RGB') for path in examples['image_path']]
    words = examples['words']
    boxes = examples['bboxes']
    word_labels = examples['ner_tags']

    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels, padding="max_length", truncation=True)
    return encoded_inputs

train_dataset = datasets['train'].map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names, features=features)
test_dataset = datasets['test'].map(preprocess_data, batched=True, remove_columns=datasets['test'].column_names, features=features)

Downloading:   0%|          | 0.00/136 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
processor.tokenizer.decode(train_dataset['input_ids'][0])

'[CLS] r & d : suggestion : date : licensee yes no 597005708 r & d quality improvement suggestion / solution form name / phone ext. : m. hamann p. harper, p. martinez 9 / 3 / 92 r & d group : j. s. wigand supervisor / manager discontinue coal retention analyses on licensee submitted product samples ( note : coal retention testing is not performed by most licensees. other b & w physical measurements as ends stability and inspection for soft spots in ciparettes are thought to be sufficient measures to assure cigarette physical integrity. the proposed action will increase laboratory productivity. ) suggested solutions ( s ) : delete coal retention from the list of standard analyses performed on licensee submitted product samples. special requests for coal retention testing could still be submitted on an exception basis. have you contacted your manager / supervisor? manager comments : manager, please contact suggester and forward comments to the quality council. qip. wp [SEP] [PAD] [PAD] [

In [13]:
print(train_dataset['labels'][0])

[-100, 0, -100, -100, 3, 3, -100, 3, -100, 5, -100, 3, 3, 0, -100, -100, -100, -100, -100, 1, -100, -100, 2, 2, 2, -100, 2, 2, 3, 4, 4, 4, -100, -100, 4, 5, -100, 6, -100, 6, -100, 6, -100, 6, -100, 6, 5, -100, 6, -100, 6, 3, -100, -100, 4, -100, 5, -100, 6, -100, 6, -100, 3, 4, 4, 5, -100, -100, -100, 6, 6, 6, 6, 6, -100, 6, 6, 6, 6, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100, -100, 6, 6, -100, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 4, 4, -100, -100, 4, 5, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100, 6, 6, 6, -100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100, 1, 2, 2, 2, 2, -100, 2, -100, 3, 4, -100, 5, -100, 6, 6, 6, -100, 6, 6, 5, 6, 6, 6, 6, -100, 0, -100, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [14]:
train_dataset.set_format(type="torch", device="cuda")
test_dataset.set_format(type="torch", device="cuda")

In [15]:
train_dataset.features.keys()

dict_keys(['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'])

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [17]:
batch = next(iter(train_dataloader))

for k,v in batch.items():
    print(k, v.shape)

image torch.Size([4, 3, 224, 224])
input_ids torch.Size([4, 512])
attention_mask torch.Size([4, 512])
token_type_ids torch.Size([4, 512])
bbox torch.Size([4, 512, 4])
labels torch.Size([4, 512])


## Train the model

In [28]:
from numpy.lib.arraysetops import isin
import numpy as np
from transformers import LayoutLMv2ForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric

model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased',
                                                         num_labels=len(label2id))

# Set id2label and label2id
model.config.id2label = id2label
model.config.label2id = label2id

# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n,v in value.items():
                    final_results[f'{key}_{n}'] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"]
        }
    

class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
        return train_dataloader
    
    def get_test_dataloader(self, test_dataset):
        return test_dataloader

args = TrainingArguments(
    output_dir="layoutlmv2-finetuned-funsd-v2",
    max_steps=2000,
    warmup_ratio=0.1,
    fp16=True
)

# Initialize out Trainer
trainer = FunsdTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/microsoft/layoutlmv2-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/882f0cab8dbb456e5b1d6e3b96e864be0cb6c2bc5d20ee88eeda10b3c0317332.a3f80b6502f00efe74a01c6a007f196803229a24224659c81c1da7c4cf5316e8
Model config LayoutLMv2Config {
  "attention_probs_dropout_prob": 0.1,
  "convert_sync_batchnorm": true,
  "coordinate_size": 128,
  "detectron2_config_args": {
    "MODEL.ANCHOR_GENERATOR.SIZES": [
      [
        32
      ],
      [
        64
      ],
      [
        128
      ],
      [
        256
      ],
      [
        512
      ]
    ],
    "MODEL.BACKBONE.NAME": "build_resnet_fpn_backbone",
    "MODEL.FPN.IN_FEATURES": [
      "res2",
      "res3",
      "res4",
      "res5"
    ],
    "MODEL.MASK_ON": true,
    "MODEL.PIXEL_STD": [
      57.375,
      57.12,
      58.395
    ],
    "MODEL.POST_NMS_TOPK_TEST": 1000,
    "MODEL.RESNETS.ASPECT_RATIOS": [
      [
        0.5,
        1.0,
    

In [29]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 149
  Num Epochs = 53
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
/usr/local/lib/python3.7/dist-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:780: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div

Step,Training Loss
500,0.905500
1000,0.212000
1500,0.079200
2000,0.046300


Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-500
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:780: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  // self.config.image_feature_pool_shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:790: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'tru

TrainOutput(global_step=2000, training_loss=0.31076879405975344, metrics={'train_runtime': 1179.9295, 'train_samples_per_second': 13.56, 'train_steps_per_second': 1.695, 'total_flos': 4234463136043008.0, 'train_loss': 0.31076879405975344, 'epoch': 52.63})

In [30]:
predictions, labels, metrics = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 50
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:780: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  // self.config.image_feature_pool_shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:790: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual

In [31]:
print(metrics)

{'test_loss': 0.8864361047744751, 'test_ANSWER_precision': 0.8011764705882353, 'test_ANSWER_recall': 0.8417799752781211, 'test_ANSWER_f1': 0.8209764918625678, 'test_ANSWER_number': 809, 'test_HEADER_precision': 0.6293103448275862, 'test_HEADER_recall': 0.6134453781512605, 'test_HEADER_f1': 0.6212765957446807, 'test_HEADER_number': 119, 'test_QUESTION_precision': 0.8470046082949308, 'test_QUESTION_recall': 0.8629107981220657, 'test_QUESTION_f1': 0.8548837209302326, 'test_QUESTION_number': 1065, 'test_overall_precision': 0.8156996587030717, 'test_overall_recall': 0.8394380331159057, 'test_overall_f1': 0.8273986152324431, 'test_overall_accuracy': 0.8148494046229279, 'test_runtime': 3.3464, 'test_samples_per_second': 14.941, 'test_steps_per_second': 2.092}
